In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys, pickle, random, glob, dill, time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from importlib import reload

top = '/Users/dfporter/pma/dataAndScripts/clip/miseq/v2all//'
os.chdir(top)

import sameRiver
import sameRiver.biotypeUtils
import sameRiver.intronExonSplit
import sameRiver.statsForCountsNB
import sameRiver.biotypeAdder
import sameRiver.biotypeDf
import sameRiver.biotypeArtist
import sameRiver.stacked_bargraph
import sameRiver.metadata.negative_metadata as negative_metadata
importlib.reload(sameRiver.biotypeAdder)
importlib.reload(sameRiver.biotypeUtils)
importlib.reload(sameRiver.countsO)
importlib.reload(sameRiver.stacked_bargraph)
importlib.reload(sameRiver.statsForCountsNB)
importlib.reload(sameRiver.biotypeArtist)
importlib.reload(sameRiver.biotypeDf)

# Load data and globals.

scheme = sameRiver.scheme.scheme('./scheme.xlsx')

randos_rbps = [
    'FBL', 'Rbfox1', 'Rbfox2', 'hnRNPC', 
    'hnRNPD', 'CELF1', 'SF3B1', 'PCBP1',
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
    'PCBP1 100P', 'PCBP1 100Q', 'PCBP1 dKH']
randos = [
    'UBA2', 'ETS2', 'CAPNS6', 'TPGS2', 
    'EPB41L5', 'CHMP3', 'CDK4', 'IDE', 
    'DCTN6', 'CCIN', 'ITPA', 
]
rbps = [
    'FBL', 'hnRNPC', 'SF3B1', 'PCBP1', 
    #'PCBP1:100P', 'PCBP1:100Q','PCBP1:dKH',
    'CELF1', 'Rbfox1', 'Rbfox2', 'hnRNPD',
    ]
# Select PCBP1s
pcbp1_names = ['PCBP1', 'PCBP1:100P', 'PCBP1:100Q', 'PCBP1:dKH', 'Gene type']


In [ ]:
# Create a palette to be used for all figures.
type_list = ['rRNA', 'ribozyme', #'7SK', 
             'processed_transcript', 'snoRNA', 'Repetitive element', 
             'tRNA', 'scRNA', 'lincRNA',
             'snRNA', 'LTR', 'protein_coding']
type_colors = sns.cubehelix_palette(n_colors=len(type_list), start=.5, rot=-2)

group_to_color = {}
for m, _t in enumerate(type_list):  # Zip wouldn't iterate the colors for some reason.
    group_to_color[_t] = type_colors[m]

In [ ]:
df = pandas.read_csv('./data/positive_counts_per_protein.txt', sep='\t', index_col=0)   
#print(df.iloc[0,:])
#df['Gene type'] = [re.sub('>0_rRNA_Hsa#rRNA', 'rRNA', x) for x in df['Gene type']]

df['Gene type'] = [re.sub('.*/.*', 'Repetitive element', x) for x in df['Gene type']]
res = df.groupby(by='Gene type').sum()
#    fraction_all_reads.loc[:,'Protein'] = [scheme.gene_from_fname(x) for x in fraction_all_reads['Protein']]
#    fraction_all_reads = fraction_all_reads.groupby(['Protein', 'Gene type'])['% reads (all RNAs)'].mean().reset_index()

res.loc['Protein', :] = [scheme.gene_from_fname(x) for x in res.columns]
#print(res)
rep_n = collections.defaultdict(int)
for col in res.columns:
    
    rep_n[res[col]['Protein']] += 1
#print(rep_n)
g = res.T.groupby(['Protein']).sum().T
for col in g.columns:
    #print(col, rep_n[col])
    g.loc[:,col] = g[col]/rep_n[col]

g = 100 * g/1E10  # Per 100 proteins, instead of per ten billion

#g = np.log10(g)
#print(g.loc['protein_coding'])
#print(g.loc['Repetitive element'])
#print(g)
#print(g)
#toplt = g.loc[['protein_coding'], :]

df = pandas.read_excel('/Users/dfporter/pma/dataAndScripts/tables/lysateExpression.xlsx')

to_expression = dict(df.groupby('Protein')['Value'].mean())
print(to_expression)

per_protein = g.loc[:, [x for x in g.columns if 'PCBP1' in x]]

per_protein = per_protein[['PCBP1', 'PCBP1:dKH', 'PCBP1:100Q', 'PCBP1:100P']]

per_cell = per_protein.copy()
for col in per_protein.columns:
    regular_col = re.sub(':', ' ', col)
    regular_col = re.sub('dKH', 'ΔKH', regular_col)
    per_cell[col] = to_expression[regular_col] * per_protein[col]

print(per_cell)

def plot_bar(g, title, ylabel=''):
    
    toplt = g.loc[['protein_coding', #'Repetitive element', 
                   'snRNA', #'snoRNA', 
                   'processed_transcript', 'rRNA',],:]
    
    toplt['Gene type'] = toplt.index
    print(toplt)
    toplt = toplt.melt(id_vars=['Gene type'])
    import seaborn as sns
    plt.clf()
    fig = plt.figure()
    ax = sns.barplot(y='Protein', x='value', data =toplt, hue='Gene type', palette=group_to_color,)
    #plt.xlim(0, 1)
    #plt.xticks(rotation=90)
    plt.ylabel(ylabel)
    fig.set_figwidth(3)
    fig.set_figheight(2)
    fig.legend().set_visible(False)
    ax.legend().remove()
    fig.savefig('/Users/dfporter/pma/dataAndScripts/clip/figs/{}.pdf'.format(title))
    plt.show()
    plt.clf()
    
    
plot_bar(per_protein, 'PCBP1_bar_XLs_per_protein', ylabel='XLs per 100 proteins')
plot_bar(per_cell, 'PCBP1_bar_XLs_per_cell', ylabel='XLs per cell (relative)')

